# Svenskt Kvinnobiografiskt lexikon 

* Wikidataproperty [P4963](https://www.wikidata.org/wiki/Property_talk:P4963)

In [39]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None)
pd.set_option('display.max_rows', None)

dftot = pd.DataFrame()
 
url = "https://ws.spraakbanken.gu.se/ws/karp/v5/minientry?q=extended%7C%7Cand%7Cnamn%7Cexists&mode=skbllinks&show=name%2Curl&sort=sorteringsnamn.sort%2Csorteringsnamn.init%2Ctilltalsnamn.sort&size=10000&resource=skbl"
r = http.request('GET', url) 
data = json.loads(r.data.decode('utf-8')) 
#print(type(data))
#print(r.info())
listSKBL = []
for row in (data["hits"]["hits"]):
    print (row["_source"]["url"])
    listSKBL.append(row["_source"]["url"])
dftot = pd.DataFrame(listSKBL)


MargitAbenius
EuphrosyneAbrahamson
AlmaAbrahamsson
IngelaMargaretaAbrahamsson
WilhelminaSofiaAcrel
KarinAdamsson
MariaAdelborg
OttiliaAdelborg
GertrudAdelborg
AmandaAdler
CeliaAdler
RutMariaAdler
MariaAdlercreutz
MaudAdlercreutz
SophieAdlersparre
SophieAdolphineAdlersparre
MarthaAdlerstrahle
EvaAdolfsson
GretaAdrian
MartaAfzelius
AnnaBrittAgnsater
AlfhildAgrell
AnnaAgriconia
FadimeSahindal
LeaAhlborn
CatharinaAhlgren
VictoriaBenedictsson
GunnelMariaAhlin
AnnaAhlstrom
AstridAhnfelt
HildaHenriettaAhnfelt
KarinAhrland
ErikaAittamaa
LissiAlandh
MargitMabelGeorginaAlbins
EvaAlexanderson
VeraAlexandrova
BirgittaAlexius0
HarrietAlfons
AnneMarieAlfvenEriksson
AliceBabsSjoblom
KarinAlin
AnnaAlinBosson
JulieAlixdelaFaye
AnnastinaAlkman
InaAlmen
RuthAlmen
SofiAlmquist
EsterAlmqvist
IngridMargaretaAlmqvist
MargaretaAlstromer
MargitAlthin
AnnCharlotteAlverfors0
FannyAlving
BarbroAlvingBang
ElinAmeen
EmmaAndelius0
EvaAnden
AstridAnderberg
AstriAndreaKristinaAndersen
AnnaLisaAndersson
BeritElisabetBi

UllaStenberg
LisetteStenberg
MariaKatrinaKarinStenberg
HedvigEleonoraStenbock
StinaStenhagen
ElsaStenhammar
FredrikaStenhammar
HilleviStenhammar
MariaStenkula
ElsaStenudd
AmelinaSterky
AnnaSterky
MarikaStiernstedt
KarinStjernholmRder
RuthStjernstedt
KarinStolpe
GustiStridsberg
UlrikaSophiavonStrussenfelt
MathildaHamilton
UlrikaStromfelt
KristinaSvantesdotterSture
SigridSture
AnnaMargaretaAnnaGretaStahle
BirgitStahlNyberg
CarolinaWilhelminaStahlberg
UlrikaEleonoraStalhammar
StinaStenhagen
MargaretaSuber
StinaSundberg
ElinSundberg
StinaSundell
AlmaSundquist
HarrietSundstrom0
BirgittaSvanberg
AnnaSvantesdotterSture
NannaSvartz
HilmaSvedal
AinaLenaLenaBerniceSvedberg
RuthAstaSvedberg
HertaSvenson
WilhelminaSvensson
ElinTeresiaAndreaSvensson
ValborgSvensson
InezSvensson
GardaSvensson
RebeckaSvensson0
RuthSvensson
SofiaSvensson
BarbroSvenssonLillBabs
ElsaKarolinaSvenssonSanfridsson
KarinSwanstrom
KristinaStinaLovisaSwartling
TeklaSwedlund
AldaTulliaBjornsdottervonSydow
ViviSylwan
AnnaSarstro

In [40]:
dftot.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495 entries, 0 to 1494
Data columns (total 1 columns):
0    1495 non-null object
dtypes: object(1)
memory usage: 11.8+ KB


In [ ]:
dftot.url.value_counts()

In [ ]:
SKBLlst = [] 
for index , value  in dftot.url.value_counts().items():
    #print(index,value)
    SKBLlst.append(index) 
SKBLtot = pd.DataFrame(SKBLlst)    
SKBLtot.info()

# Wikidata

In [ ]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT * WHERE {
?item wdt:P31 wd:Q5.
?item wdt:P4963 ?skbl
} order by ?skbl"""


def get_results(endpoint_url, query):
    user_agent = "User:salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)
 
lst = []
for result in results["results"]["bindings"]:
    lst.append((result["skbl"]["value"]))
WDtot = pd.DataFrame(lst) 
WDtot.info()

In [ ]:
WDtot

In [ ]:
WDtot.sort_index(inplace=True) 
SKBLtot.sort_index(inplace=True) 
  
diff = pd.concat([WDtot, SKBLtot]) 
diff = diff.reset_index(drop=True)
diff.info()


In [ ]:
WDtot.info()

In [ ]:
SKBLtot.info()